In [ ]:
## load dependencies
import os
import time
import copy
import torch 
import random
import importlib
import numpy as np
import pandas as pd
import casadi
from math import sqrt
import torch.nn as nn
from sklearn.svm import SVR 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Hand made classes
from src import DataPreprocessing as dp
from src import Model_PINN_casadi as pinn_casadi
from src import plotter

# Organize this
device = torch.device("cpu")
torch.manual_seed(4)
np.random.seed(4) 
random.seed(4)

#        [  0    ,   1      ,     2      ,     3   ,    4     ,   5    ,   6   ,    7     ,     8     ,  9    ,   10    ,  11 , 12  ,  13   ,  14 ,  15     , 16 ]
output = ['O2(X)','O2(a1Dg)', 'O2(b1Sg+)','O2(Hz)','O2(+,X)', 'O(3P)','O(1D)','O(+,gnd)', 'O(-,gnd)','O3(X)','O3(exc)', 'Tg','Tnw','Red_E', 'vd', 'E_mean','ne']

input = ['P','I', 'R']
fraction_train = 0.8
fraction_test = 0.1
fraction_val = 0.1
runLR = 0
runSamplingComparison = 1
runNN = 1
runSA = 1
apply_kfolds = 1
optimize_svr = 0
apply_PCA = 0
apply_LHS = 0
apply_PAS = 0
apply_grid_search = 0
apply_EDA = 1

# **1. Data Visualization & Pre-Processing**

   ## **1.1. Import data from .txt file and visualize it**

In [ ]:
importlib.reload(dp)
importlib.reload(plotter)

# Define the filename
filename_initial = '/Users/matildevalente/Documents/PINN_Paper/LTP_System/Datasets/Morris/p10/3input_1000.txt'
delimiter= ' '

# Extract Data From File
full_dataset = dp.LoadDataset(filename_initial)
y_data = full_dataset.y
X_data = full_dataset.x
df = pd.read_csv(filename_initial, delimiter= delimiter, header=None, names= input + output)

# Plot histogram to visualize the inputs
num_bins = 10
num_bins, min_I, max_I, col_name, x_label, title = 50, 0, 0.055, "I", 'Values in Column I', 'Histogram of Column I'

hist, bin_edges = plotter.Histogram_3inputs(df, min_I, max_I, num_bins, col_name, x_label, title)

# Example usage with dummy data
plotter.plot_pie_chart_species_densities(y_data)


## **1.2. Preprocess data: min max scaling and log-transform**

In [ ]:
# DATA DEFINITION  ------------------------------------------------------------------------------------------
# Initialize Plots and define loss func
importlib.reload(dp)
importlib.reload(plotter)

# Define Threshold
skew_threshold_up = 3
skew_threshold_down = 0#0.09

# Create data_prepocessed object
path = '/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/Histograms/'
data_prepocessed = dp.DataPreprocessor(output, input, fraction_train, fraction_val, skew_threshold_down, skew_threshold_up)
data_prepocessed.setup_dataset(X_data, y_data, apply_EDA, path)
data_prepocessed_copy = copy.deepcopy(data_prepocessed)

## **1.3. Visualize the amplitute of different input and output features**

In [ ]:
importlib.reload(plotter)
plotter.plot_feature_amplitutes(y_data)

# **2. Implement Data-Driven NN Model**

## **2.1. Define the Model**

In [ ]:
# DO NOT RUN
importlib.reload(pinn_casadi)
import torch.nn.functional as F

# NEURAL NETWORK  ---------------------------------------------------------------------------------------------------------------------------------
# 1. Define Hyperparameters
print_flag = 1
show_plots = 0
batch_size = 10
lrn_rate = 0.0001
epoch_window = 50
max_epochs = 500
ep_log_interval = 1
training_threshold=0.0000001
loss_func, loss_func_data, loss_func_physics = torch.nn.MSELoss(), torch.nn.MSELoss(), torch.nn.MSELoss()

#PINN parameters - L_1: pressure; L_2: current; L_3: zero charge - relative weigth 
lambda_physics = [0, 0, 0]      

# Activation Functions to search from
activation_functions = [F.leaky_relu] #[F.leaky_relu, F.relu, F.tanh, F.sigmoid]

# Base architecture & Number of architectures to generate
base_architecture = [10, 10,10]
num_architectures = 75

# Get the best architecture
W = np.eye(len(data_prepocessed.output))
model = (pinn_casadi.PhysicsAwareNet(base_architecture, lrn_rate , batch_size, 0, lambda_physics, activation_functions, W).to(device)).to(torch.double) 
data_prepocessed_copy = copy.deepcopy(data_prepocessed)

## **2.2. Determine the Optimal Architecture Using Random Search**

In [ ]:
#min_val_loss_row = model.getArchitecture(data_prepocessed_copy, num_architectures, base_architecture, ep_log_interval, loss_func_data, loss_func_physics, training_threshold, max_epochs, epoch_window, show_plots)

In [ ]:
#print("Best Architecture: ", min_val_loss_row['architecture'])
#print("Activation Functions: ", min_val_loss_row['activ_func'])

## **2.3. Train the model with the chosen architecture**

In [ ]:
importlib.reload(pinn_casadi)
importlib.reload(dp)
importlib.reload(plotter)
import torch.nn.functional as F

print_flag = 0

# 1. Define Hyperparameters
max_epochs =  500
hidden_size_arr = [451, 315, 498, 262] #min_val_loss_row['architecture'] 
activation_func = [F.leaky_relu,F.leaky_relu,F.leaky_relu,F.leaky_relu] #min_val_loss_row['activ_func']

# number of models in the bootstrap aggregation algorithm
num_models = 50  

# 2. Train Model
myplot_nn = pinn_casadi.MyPlots()
data_prepocessed_copy = copy.deepcopy(data_prepocessed)
val_loader = torch.utils.data.DataLoader(data_prepocessed_copy.val_data, batch_size=batch_size, shuffle=True) # set to True
nn_models, epoch_loss_arr_train, epoch_loss_arr_val, best_seed_pinn, epoch_loss_train_physics_sanity  = pinn_casadi.getTrainedPinns(val_loader,data_prepocessed_copy, num_models, hidden_size_arr, lrn_rate, batch_size, 0, lambda_physics, activation_func, epoch_window,print_flag, ep_log_interval, myplot_nn, loss_func_data, loss_func_physics, training_threshold, max_epochs)


In [ ]:
# 3. Plot Loss Curve
importlib.reload(plotter)

path = '/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/model_training_loss_curves/'
plotter.LossCurvePlotNN(myplot_nn, path, " ", num_models)


## **2.4. See Performance of NN and Compare it to Projection-Based PINN**

In [ ]:
importlib.reload(pinn_casadi)
importlib.reload(plotter)

# 1. Path that contains experimental dataset
file_name = "/Users/matildevalente/Documents/PINN_Paper/LTP_System/Datasets/Morris/p10/3input_300.txt"

# 2. Path where the plots are to be stored
save_path = "/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/bar_plot_NN/"

# 3. Get results for identity matrix
model_name = "NN"
rse_pinn_3_laws, sem_pinn_3_laws, rse_nn, sem_nn = plotter.get_plot_W_matrix(torch.eye(17), file_name, data_prepocessed, nn_models, num_models, model_name, num_bars = 8, path = save_path)    
plotter.get_plot_W_matrix(torch.eye(17), file_name, data_prepocessed, nn_models, num_models, model_name, num_bars = 5, path = save_path)    
plotter.get_plot_W_matrix(torch.eye(17), file_name, data_prepocessed, nn_models, num_models, model_name, num_bars = 2, path = save_path)    


# **3. Train the loss based PINN**

## **3.1. Determine the optimal lambda parameter**

In [ ]:
importlib.reload(pinn_casadi)

myplot_standard_PINN = pinn_casadi.MyPlots()
num_pinns_lambda_optimization = 1
max_epochs_lambda_optimization = 60
print_flag = 0
num_iterations = 1
best_lambda_physics, list_val_losses, list_lambdas = pinn_casadi.optimize_lambda_physics(num_pinns_lambda_optimization, max_epochs_lambda_optimization, base_architecture, lrn_rate, batch_size, activation_functions, W, data_prepocessed_copy, hidden_size_arr, activation_func, epoch_window, print_flag, ep_log_interval, myplot_nn, loss_func_data, loss_func_physics, training_threshold, num_iterations)

# Create a DataFrame
df_lambda_optimization = pd.DataFrame(list_lambdas, columns=['lambda_physics_1', 'lambda_physics_2', 'lambda_physics_3'])
df_lambda_optimization['lambda_data'] = 1 - (df_lambda_optimization['lambda_physics_1'] + df_lambda_optimization['lambda_physics_2'] + df_lambda_optimization['lambda_physics_2'])
df_lambda_optimization['val_loss'] = list_val_losses

# Save dataframe to computer in a CSV file
full_path = "/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/lambda_optimization/lambda_optimization.csv"
df_lambda_optimization.to_csv(full_path, index=False)
df_lambda_optimization

## **3.2. Train the loss based PINN Model**

In [ ]:
print(lrn_rate)

In [ ]:
importlib.reload(pinn_casadi)
import torch.nn.functional as F
importlib.reload(dp)
importlib.reload(plotter)
import itertools

# NEURAL NETWORK  ---------------------------------------------------------------------------------------------------------------------------------
#PINN parameters
#print("Best lambda = ", best_lambda_physics)
#lambda_physics =  [] #best_lambda_physics    # [0.01036426, 0.53862728, 0.00301796] [0.03063382, 0.51784745, 0.0035539 ] # # L_1: pressure; L_2: current; L_3: zero charge - relative weigth 


def generate_lambda_physics_values(N, options):
        
    # Generate all possible combinations of the remaining values
    all_combinations = list(itertools.product(options, repeat=3))
    
    # Filter combinations to ensure the sum is less than 1
    valid_sublists = [
        [comb[0], comb[1], comb[2]] 
        for comb in all_combinations 
        if comb[0] + comb[1] + comb[2] < 1
    ]
    
    # Return only the first N sublists
    return valid_sublists


# Example predetermined set of options
options = np.linspace(0, 0.999, 10)
options = np.logspace(np.log(0.0001), 0, 10)
N = len(list(itertools.product(options, repeat=3)))
print(options)

# Generate a random list of lists with 4 sublists, each of length 4
lambda_physics_list = generate_lambda_physics_values(N, options=options)
print(lambda_physics_list)
val_loss_list, train_LoKI_loss = [], []
print_flag = 0

#
"""for lambda_physics_list in lambda_physics_list:

    # 2. Get the best architecture
    model_pinn_standard = (pinn_casadi.PhysicsAwareNet(base_architecture, lrn_rate , batch_size, 0, lambda_physics, activation_functions, W).to(device)).to(torch.double) 

    # 3. Train Model
    myplot_pinn_standard = pinn_casadi.MyPlots()
    data_prepocessed_copy = copy.deepcopy(data_prepocessed)
    val_loader = torch.utils.data.DataLoader(data_prepocessed_copy.val_data, batch_size=batch_size, shuffle=True) # set to True

    pinns_standard, epoch_loss_arr_train, epoch_loss_arr_val, best_seed_pinn, epoch_loss_train_physics_sanity  = pinn_casadi.getTrainedPinns(val_loader,data_prepocessed_copy, num_models, hidden_size_arr, lrn_rate, batch_size, 0, lambda_physics, activation_func, epoch_window,print_flag, ep_log_interval, myplot_pinn_standard, loss_func_data, loss_func_physics, training_threshold, max_epochs)

    # 4. Append the value of the validation loss
    val_loss_list.append(epoch_loss_arr_val[-1])
    train_LoKI_loss.append(epoch_loss_train_physics_sanity[-1])

#
min_index = val_loss_list.index(min(val_loss_list))
print(f"Best lambda: {lambda_physics_list[min_index]}")

# Create the plot
plt.figure(figsize=(5, 5), dpi=300)  
plt.plot(np.array(lambda_physics_list)[:,0], val_loss_list, marker='o', label=r'$L_{VAL}$')
plt.plot(np.array(lambda_physics_list)[:,0], train_LoKI_loss, marker='o', label=r'$L$(y, $y_{LoKI}$)')
plt.xlabel(r'$\lambda_{Physics}$')
plt.ylabel(r'$Loss$')
plt.legend()
plt.show()"""


In [ ]:
importlib.reload(pinn_casadi)
import torch.nn.functional as F
importlib.reload(dp)
importlib.reload(plotter)

# NEURAL NETWORK  ---------------------------------------------------------------------------------------------------------------------------------
#PINN parameters
#best_lambda_physics = lambda_physics_list[min_index]
#print("Best lambda = ", best_lambda_physics)
#lambda_physics =  best_lambda_physics    # [0.01036426, 0.53862728, 0.00301796] [0.03063382, 0.51784745, 0.0035539 ] # # L_1: pressure; L_2: current; L_3: zero charge - relative weigth 
lambda_physics = [0.01036426, 0.53862728, 0.00301796]
lambda_physics = [0.0001036426, 0.0053862728, 0.0000301796]

print_flag = 1

# 2. Get the best architecture
model_pinn_standard = (pinn_casadi.PhysicsAwareNet(base_architecture, lrn_rate , batch_size, 0, lambda_physics, activation_functions, W).to(device)).to(torch.double) 

# 3. Train Model
myplot_pinn_standard = pinn_casadi.MyPlots()
data_prepocessed_copy = copy.deepcopy(data_prepocessed)
val_loader = torch.utils.data.DataLoader(data_prepocessed_copy.val_data, batch_size=batch_size, shuffle=True) # set to True

#pinns_standard, epoch_loss_arr_train, epoch_loss_arr_val, best_seed_pinn, epoch_loss_train_physics_sanity  = pinn_casadi.getTrainedPinns(val_loader,data_prepocessed_copy, num_models, hidden_size_arr, lrn_rate, batch_size, 0, lambda_physics, activation_func, epoch_window,print_flag, ep_log_interval, myplot_pinn_standard, loss_func_data, loss_func_physics, training_threshold, max_epochs)
pinns_standard, epoch_loss_arr_train, epoch_loss_arr_val, best_seed_pinn, epoch_loss_train_physics_sanity  = pinn_casadi.getTrainedPinns(val_loader,data_prepocessed_copy, 2, hidden_size_arr, lrn_rate, batch_size, 0, lambda_physics, activation_func, epoch_window,print_flag, ep_log_interval, myplot_pinn_standard, loss_func_data, loss_func_physics, training_threshold, max_epochs)




In [ ]:
# 1. Make copies to avoid modifying the value
importlib.reload(plotter)
importlib.reload(pinn_casadi)
dataset_file_name = "/Users/matildevalente/Documents/PINN_Paper/LTP_System/Datasets/Morris/p10/3input_300.txt"

# 2. Plot of the predicted vs target pressure, discharge current and electron density for the NN model and the Projection Model
save_image_path = "/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/compliance_with_physical_laws_NN/"
model_name = "NN"
error_P_NN_before_proj, error_I_NN_before_proj, error_ne_NN_before_proj, error_P_NN_after_proj, error_I_NN_after_proj, error_ne_NN_after_proj=plotter.Pred_vs_Target_Laws_from_file(nn_models, data_prepocessed_copy, output, num_models, input, W, dataset_file_name, save_image_path, model_name)

# 3. Plot of the predicted vs target pressure, discharge current and electron density for the Standard PINN model and the Projection Model
save_image_path = "/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/compliance_with_physical_laws_standard_PINN/"
model_name = "Standard PINN"
error_P_standard_PINN_before_proj, error_I_standard_PINN_before_proj, error_ne_standard_PINN_before_proj, error_P_standard_PINN_after_proj, error_I_standard_PINN_after_proj, error_ne_standard_PINN_after_proj=plotter.Pred_vs_Target_Laws_from_file(pinns_standard, data_prepocessed_copy, output, num_models, input, W, dataset_file_name, save_image_path, model_name)


In [ ]:
# 3. Plot Loss Curve
importlib.reload(plotter)
path = '/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/model_training_loss_curves/'
plotter.LossCurvePlotPINN(myplot_pinn_standard, path, " ", num_models)

## **3.2. See Performance of Loss Based PINN and Compare it to Projection-Based PINN**

# FUNCOES POR IMPLEMENTAR!!!

In [ ]:
importlib.reload(dp)
importlib.reload(plotter)
importlib.reload(pinn_casadi)

# 1. Make copies to avoid modifying the value
nn_models_safe = copy.deepcopy(nn_models)
data_prepocessed_copy = copy.deepcopy(data_prepocessed)


######## POR IMPLEMENTAR
# 3. Plot of each output as a function of discharge current for a pressure of 2 Torr and a radius of the reactor of 12 mm 
path_plots_outputs = '/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/outputs_vs_current_single_P/'
dataset_exp = '/Users/matildevalente/Documents/PINN_Paper/LTP_System/Datasets/ConstPressure/3input_40_3Torr.txt'
#mean_rse_arr_NN, mean_rse_arr_PINN = plotter.Plot_Output_vs_Current_single_P(data_prepocessed_copy , dataset_exp, nn_models_safe, num_models, path_plots_outputs, log_y_scale = 0, constraint_func=pinn_casadi.compositeContraintFunc, W=torch.eye(17))



## Barplots comparing errors when applying different constraints

In [ ]:
importlib.reload(pinn_casadi)
importlib.reload(plotter)

# 1. Extract experimental dataset
file_name = "/Users/matildevalente/Documents/PINN_Paper/LTP_System/Datasets/Morris/p10/3input_300.txt"

# 2. Path where the plots are to be stored
save_path = "/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/bar_plot_standard_PINN/"

# 3. Get results for identity matrix
model_name = "Standard PINN"
plotter.get_plot_W_matrix(torch.eye(17), file_name, data_prepocessed, pinns_standard, num_models, model_name, num_bars = 8, path = save_path)    
plotter.get_plot_W_matrix(torch.eye(17), file_name, data_prepocessed, pinns_standard, num_models, model_name, num_bars = 5, path = save_path)    
plotter.get_plot_W_matrix(torch.eye(17), file_name, data_prepocessed, pinns_standard, num_models, model_name, num_bars = 2, path = save_path)   
model_name = "NN"
plotter.get_plot_W_matrix(torch.eye(17), file_name, data_prepocessed, nn_models, num_models, model_name, num_bars = 2, path = save_path)     

In [ ]:
importlib.reload(plotter)

############# PLOT DESCRIPTION #################
save_path = "/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/bar_plot_NN_and_standard_PINN/"
dataset_name = '/Users/matildevalente/Documents/PINN_Paper/LTP_System/Datasets/Morris/p10/3input_300.txt'
RSE_proj_standard_PINN, RSE_not_proj_standard_PINN = plotter.get_plot_W_matrix_casadi(torch.eye(17), file_name, data_prepocessed, pinns_standard, num_models, num_bars = 2)    
RSE_proj_NN, RSE_not_proj_NN = plotter.get_plot_W_matrix_casadi(np.eye(17), dataset_name, data_prepocessed_copy, nn_models, num_models, num_bars=2)

plotter.PINN_NN_Proj_comparison(RSE_not_proj_standard_PINN, RSE_proj_standard_PINN, RSE_proj_NN, RSE_not_proj_NN, save_path)


## Compliance with physical laws analysis

In [ ]:
# 1. Make copies to avoid modifying the value
importlib.reload(plotter)
importlib.reload(pinn_casadi)
dataset_file_name = "/Users/matildevalente/Documents/PINN_Paper/LTP_System/Datasets/Morris/p10/3input_300.txt"

# 2. Plot of the predicted vs target pressure, discharge current and electron density for the NN model and the Projection Model
save_image_path = "/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/compliance_with_physical_laws_NN/"
model_name = "NN"
error_P_NN_before_proj, error_I_NN_before_proj, error_ne_NN_before_proj, error_P_NN_after_proj, error_I_NN_after_proj, error_ne_NN_after_proj=plotter.Pred_vs_Target_Laws_from_file(nn_models, data_prepocessed_copy, output, num_models, input, W, dataset_file_name, save_image_path, model_name)

# 3. Plot of the predicted vs target pressure, discharge current and electron density for the Standard PINN model and the Projection Model
save_image_path = "/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/compliance_with_physical_laws_standard_PINN/"
model_name = "Standard PINN"
error_P_standard_PINN_before_proj, error_I_standard_PINN_before_proj, error_ne_standard_PINN_before_proj, error_P_standard_PINN_after_proj, error_I_standard_PINN_after_proj, error_ne_standard_PINN_after_proj=plotter.Pred_vs_Target_Laws_from_file(pinns_standard, data_prepocessed_copy, output, num_models, input, W, dataset_file_name, save_image_path, model_name)

# Row names
row_names = ['P Error', 'I Error', 'ne Error']

# Create a DataFrame
df = pd.DataFrame({
    'NN Model Before Projection ': [error_P_NN_before_proj, error_I_NN_before_proj, error_ne_NN_before_proj],
    'NN Model After Projection': [error_P_NN_after_proj, error_I_NN_after_proj, error_ne_NN_after_proj],
    'Loss Based Model Before Projection': [error_P_NN_after_proj, error_I_NN_after_proj, error_ne_NN_after_proj],
    'Loss Based Model After Projection': [error_P_NN_after_proj, error_I_NN_after_proj, error_ne_NN_after_proj]
}, index=row_names)

# Display the DataFrame
print(df)

## Analysis of the Pressure and Discharge Current Dependencies

In [ ]:
importlib.reload(dp)
importlib.reload(plotter)
importlib.reload(pinn_casadi)

# 1. Make copies to avoid modifying the value
nn_models_safe = copy.deepcopy(nn_models)
data_prepocessed_copy = copy.deepcopy(data_prepocessed)

# 2. Plot of each output as a function of pressure for a discharge current of 30 mA and a radius of the reactor of 12 mm 
path_plots_outputs = '/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/outputs_vs_pressure_single_I/'
dataset_exp = '/Users/matildevalente/Documents/PINN_Paper/LTP_System/Datasets/ConstCurrent/3input_50_30mA.txt'
mean_rse_arr_NN, mean_rse_arr_PINN = plotter.Plot_Output_vs_Pressure_single_I(data_prepocessed_copy , dataset_exp, nn_models_safe, num_models, path_plots_outputs, log_y_scale = 0, constraint_func=pinn_casadi.compositeContraintFunc, W=torch.eye(17))

# 5. Plot of each output as a function of pressure for discharge currents of 30, 50, and 70 mA and a radius of the reactor of 12 mm 
file_names = []
currents = [10,20,30, 40]
path_plots_outputs = '/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/outputs_vs_pressure_multiple_I/'
for current in currents:
    name = '/Users/matildevalente/Documents/PINN_Paper/LTP_System/Datasets/ConstCurrent/3input_50_'+str(current)+'mA.txt'
    file_names.append(name)
plotter.Plot_Output_vs_Pressure_multi_I(data_prepocessed_copy, file_names, nn_models_safe, num_models, path_plots_outputs)

# 4. Plot of each output as a function of discharge current for pressures of 3, 5, and 7 Torr and a radius of the reactor of 12 mm 
file_names = []
pressures = [3,5,7]
path_plots_outputs = '/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/outputs_vs_current_multiple_P/'
for pressure in pressures:
    name = '/Users/matildevalente/Documents/PINN_Paper/LTP_System/Datasets/ConstPressure/3input_40_'+str(pressure)+'Torr.txt'
    file_names.append(name)
plotter.Plot_Output_vs_Current_multi_P(data_prepocessed_copy, file_names, nn_models_safe, num_models, path_plots_outputs, pinn_casadi.compositeContraintFunc, W, dont_apply_projection = 0)

# Analysis of the best seed instead of aggregated model

In [ ]:
"""# 6. Similar to (2) but in the computation we use the best seed model instead of the average of seeds
mean_rse_arr_NN, mean_rse_arr_PINN = plotter.Plot_Output_vs_Pressure_single(data_prepocessed_copy , dataset_exp, [best_seed_pinn], 1, path_plots_outputs, log_y_scale = 0, constraint_func=pinn_casadi.compositeContraintFunc, W=torch.eye(17))

# 7. Similar to (3) but in the computation we use the best seed model instead of the average of seeds
dataset_file_name = "/Users/matildevalente/Documents/PINN_Paper/LTP_System/Datasets/Morris/p10/3input_300.txt"
save_path = "/Users/matildevalente/Documents/PINN_Paper/LTP_System/Final_Plots/compliance_with_physical_laws_single_seed/"
plotter.Pred_vs_Target_Laws_from_file([best_seed_pinn], data_prepocessed_copy, output, 1, input, W, dataset_file_name, save_path)"""